In [1]:
import tensorflow as tf
import keras
import numpy as np
import pandas as pd
import cv2
import shutil
import matplotlib.pyplot as plt
import os
import time
import mediapipe as mp
import smtplib
import shutil
import random


from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.applications import MobileNetV2
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.models import load_model
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.layers import RandomFlip, RandomRotation, RandomBrightness, RandomContrast, RandomZoom
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint


In [5]:
# Define paths 
dataset_train_path ="C:\\Users\\avish\\OneDrive\\Desktop\\Facial_Issues_Detector\\train"
dataset_test_path = "C\:\\Users\\\avish\\OneDrive\\Desk\top\\Facial_Issues_Detector\\test"
labels_file = "C:\\Users\\avish\\OneDrive\\Desktop\\Facial_Issues_Detector\\train\\_classes.csv"

In [7]:
# Load labels
df = pd.read_csv(labels_file)
df.columns = df.columns.str.strip()  

In [9]:
# Print first few rows to verify data
print(df.head())
print(df.columns)

                                            filename  acne  eksim  herpes  \
0   ro32_jpg.rf.f8ac49360e8b2e405844172628d4889d.jpg     0      0       0   
1   ac33_JPG.rf.f6242c214a107b9d2b28b9e1432a0d7c.jpg     1      0       0   
2   ro31_jpg.rf.f4fbd7f5455f72244ca7b8e0d49be092.jpg     0      0       0   
3   he23_jpg.rf.fc8b15e945ebeb18c658a03f2fd4835b.jpg     0      0       1   
4  pa100_jpeg.rf.f543e4609835ec62e8b0dd4fd862e1d8...     0      0       0   

   panu  rosacea  
0     0        1  
1     0        0  
2     0        1  
3     0        0  
4     1        0  
Index(['filename', 'acne', 'eksim', 'herpes', 'panu', 'rosacea'], dtype='object')


In [11]:
dataset_train_path = r"C:\Users\avish\OneDrive\Desktop\Facial_Issues_Detector\train"
dataset_test_path = r"C:\Users\avish\OneDrive\Desktop\Facial_Issues_Detector\test"

In [13]:
print("✅ Train Path:", dataset_train_path)
print("✅ Test Path:", dataset_test_path)

✅ Train Path: C:\Users\avish\OneDrive\Desktop\Facial_Issues_Detector\train
✅ Test Path: C:\Users\avish\OneDrive\Desktop\Facial_Issues_Detector\test


In [15]:
categories = ['acne', 'eksim', 'herpes', 'panu', 'rosacea']

In [17]:
# Ensure output folders exist
for category in categories:
    os.makedirs(os.path.join(dataset_train_path, category), exist_ok=True)
    os.makedirs(os.path.join(dataset_test_path, category), exist_ok=True)

print("✅ Folders successfully created!")

✅ Folders successfully created!


In [19]:
# Load the CSV file
labels_file ="C:\\Users\\avish\\OneDrive\\Desktop\\Facial_Issues_Detector\\train\\_classes.csv"

In [21]:
df = pd.read_csv(labels_file)
df.head(10)

,filename,acne,eksim,herpes,panu,rosacea
0,ro32_jpg.rf.f8ac49360e8b2e405844172628d4889d.jpg,0,0,0,0,1
1,ac33_JPG.rf.f6242c214a107b9d2b28b9e1432a0d7c.jpg,1,0,0,0,0
2,ro31_jpg.rf.f4fbd7f5455f72244ca7b8e0d49be092.jpg,0,0,0,0,1
3,he23_jpg.rf.fc8b15e945ebeb18c658a03f2fd4835b.jpg,0,0,1,0,0
4,pa100_jpeg.rf.f543e4609835ec62e8b0dd4fd862e1d8...,0,0,0,1,0
5,ro60_jpg.rf.f870e122c5a2e024c804cd5b62fc8eec.jpg,0,0,0,0,1
6,he10_jpg.rf.fcaacfa95845eac249779b644b3ebe04.jpg,0,0,1,0,0
7,pa74_jpg.rf.fd7235fceedd735b9f914bc78ee213e9.jpg,0,0,0,1,0
8,he88_jpg.rf.fcabeb1e3a694e300c6b2a0479bff970.jpg,0,0,1,0,0
9,ro13_jpg.rf.f868ec6994ed441577eeb5e9a613ac8c.jpg,0,0,0,0,1


In [23]:
# Display the column names
print("Columns in DataFrame:", df.columns.tolist())

Columns in DataFrame: ['filename', ' acne', ' eksim', ' herpes', ' panu', ' rosacea']


In [25]:
# Remove leading and trailing whitespaces from column names
df.columns = df.columns.str.strip()

In [27]:
# Move images based on labels
for index, row in df.iterrows():
    image_name = row['filename']  # Image name column

    for category in categories:
        if category in row and row[category] == 1:  # Check if category column exists and is marked as this category
            image_train_path = os.path.join(dataset_train_path, image_name)
            image_test_path = os.path.join(dataset_test_path, image_name)

            if os.path.exists(image_train_path):
                shutil.move(image_train_path, os.path.join(dataset_train_path, category, image_name))
                print(f"✅ Moved {image_name} to {category} (Train)")
            elif os.path.exists(image_test_path):
                shutil.move(image_test_path, os.path.join(dataset_test_path, category, image_name))
                print(f"✅ Moved {image_name} to {category} (Test)")
            else:
                print(f"⚠️ Image {image_name} not found!")

            break  # Stop after first match

print("✅ Dataset successfully organized!")


✅ Moved ro32_jpg.rf.f8ac49360e8b2e405844172628d4889d.jpg to rosacea (Train)
✅ Moved ac33_JPG.rf.f6242c214a107b9d2b28b9e1432a0d7c.jpg to acne (Train)
✅ Moved ro31_jpg.rf.f4fbd7f5455f72244ca7b8e0d49be092.jpg to rosacea (Train)
✅ Moved he23_jpg.rf.fc8b15e945ebeb18c658a03f2fd4835b.jpg to herpes (Train)
✅ Moved pa100_jpeg.rf.f543e4609835ec62e8b0dd4fd862e1d8.jpg to panu (Train)
✅ Moved ro60_jpg.rf.f870e122c5a2e024c804cd5b62fc8eec.jpg to rosacea (Train)
✅ Moved he10_jpg.rf.fcaacfa95845eac249779b644b3ebe04.jpg to herpes (Train)
✅ Moved pa74_jpg.rf.fd7235fceedd735b9f914bc78ee213e9.jpg to panu (Train)
✅ Moved he88_jpg.rf.fcabeb1e3a694e300c6b2a0479bff970.jpg to herpes (Train)
✅ Moved ro13_jpg.rf.f868ec6994ed441577eeb5e9a613ac8c.jpg to rosacea (Train)
✅ Moved ek46_JPG.rf.fce062500203d4ef89c249ea4dba9399.jpg to eksim (Train)
✅ Moved ac1_jpg.rf.f534737a56ad269256b933fb12200dee.jpg to acne (Train)
✅ Moved ro100_jpg.rf.fd9b149ca6320b554f3208262b8a4f79.jpg to rosacea (Train)
✅ Moved ek19_jpg.rf.fddedb4

In [29]:
print(df.columns)

Index(['filename', 'acne', 'eksim', 'herpes', 'panu', 'rosacea'], dtype='object')


In [31]:
categories = ['acne', 'eksim', 'herpes', 'panu', 'rosacea']

In [33]:
print("Cleaned column names:", df.columns)

Cleaned column names: Index(['filename', 'acne', 'eksim', 'herpes', 'panu', 'rosacea'], dtype='object')


In [35]:
for category in categories:
    train_files = os.listdir(os.path.join(dataset_train_path, category))
    test_files = os.listdir(os.path.join(dataset_test_path, category))
    print(f"📂 {category}: {len(train_files)} training images, {len(test_files)} test images")

📂 acne: 231 training images, 0 test images
📂 eksim: 249 training images, 0 test images
📂 herpes: 249 training images, 0 test images
📂 panu: 243 training images, 0 test images
📂 rosacea: 225 training images, 0 test images


In [37]:
test_split_ratio = 0.2  # 20% test, 80% train

for category in categories:
    train_category_path = os.path.join(dataset_train_path, category)
    test_category_path = os.path.join(dataset_test_path, category)

    # Get all images in the training folder
    all_images = os.listdir(train_category_path)
    total_images = len(all_images)

    # Randomly select 20% of images for testing
    test_images = random.sample(all_images, int(test_split_ratio * total_images))

    for image in test_images:
        src_path = os.path.join(train_category_path, image)
        dest_path = os.path.join(test_category_path, image)
        shutil.move(src_path, dest_path)  # Move image to test folder

    print(f"✅ {len(test_images)} images moved to {category} test set!")

print("✅ Dataset successfully split into Train and Test sets!")

✅ 46 images moved to acne test set!
✅ 49 images moved to eksim test set!
✅ 49 images moved to herpes test set!
✅ 48 images moved to panu test set!
✅ 45 images moved to rosacea test set!
✅ Dataset successfully split into Train and Test sets!


In [39]:
for category in categories:
    train_files = os.listdir(os.path.join(dataset_train_path, category))
    test_files = os.listdir(os.path.join(dataset_test_path, category))
    print(f"📂 {category}: {len(train_files)} training images, {len(test_files)} test images")

📂 acne: 185 training images, 46 test images
📂 eksim: 200 training images, 49 test images
📂 herpes: 200 training images, 49 test images
📂 panu: 195 training images, 48 test images
📂 rosacea: 180 training images, 45 test images


In [41]:
print("Test Images Found:", len(os.listdir(dataset_test_path)))

Test Images Found: 105


In [43]:
# Load the test CSV file
test_labels_file = r"C:\Users\avish\OneDrive\Desktop\Facial_Issues_Detector\test\_classes.csv"

In [45]:
df_test = pd.read_csv(test_labels_file)
df_test.columns = df_test.columns.str.strip()  # Remove spaces

In [47]:
# Define categories
categories = ['acne', 'eksim', 'herpes', 'panu', 'rosacea']

In [49]:
# Move Test Images
for index, row in df_test.iterrows():
    image_name = row['filename']
    for category in categories:
        if row[category] == 1:
            image_test_path = os.path.join(dataset_test_path, image_name)
            if os.path.exists(image_test_path):
                shutil.move(image_test_path, os.path.join(dataset_test_path, category, image_name))
                print(f"✅ Moved {image_name} to {category} (Test)")
            else:
                print(f"⚠️ Test Image {image_name} not found!")
            break  # Stop checking after first match

print("✅ All test images sorted successfully!")

✅ Moved ek52_jpg.rf.17daf9996a95d15c97fc597ba5ac71a1.jpg to eksim (Test)
✅ Moved he14_jpg.rf.255b92dfa031400a8d8440bc0a53a2b9.jpg to herpes (Test)
✅ Moved pa25_png.rf.1f952e2878c407c3f131d3b7debd5378.jpg to panu (Test)
✅ Moved pa79_jpg.rf.2a1550f66a82ab3f207781ed1d69f683.jpg to panu (Test)
✅ Moved ek82_jpg.rf.47ba2a11cf14fbe9087087bfcc231e19.jpg to eksim (Test)
✅ Moved ac29_JPG.rf.4a284c4088aefb0a8b879026d516ba7b.jpg to acne (Test)
✅ Moved ac4_jpg.rf.393269f2ac83440b750122eff55f5d23.jpg to acne (Test)
✅ Moved n19_JPG.rf.48d44d54530949ebe1d288dab70d65b1.jpg to acne (Test)
✅ Moved he87_jpg.rf.4b111d206bce8698b03d770ea8a27f97.jpg to herpes (Test)
✅ Moved ac54_JPG.rf.247311e2ba14e74642a6de35101e8067.jpg to acne (Test)
✅ Moved he17_jpg.rf.26533c71559a31d668f9fb3db69c08a5.jpg to herpes (Test)
✅ Moved ac41_JPG.rf.5b15f290eeadb669929467a64fccd1bd.jpg to acne (Test)
✅ Moved ro18_jpg.rf.5a0fa11f64b092e421dacd2adf205bce.jpg to rosacea (Test)
✅ Moved ek8_JPG.rf.5a14a9d57c7aa8e60b99e30e5d99c839.jpg

In [51]:
for category in categories:
    test_category_path = os.path.join(dataset_test_path, category)
    test_files = os.listdir(test_category_path)
    print(f"📂 {category}: {len(test_files)} test images")

📂 acne: 69 test images
📂 eksim: 65 test images
📂 herpes: 66 test images
📂 panu: 66 test images
📂 rosacea: 70 test images


In [53]:
# Count images in the root train folder (before they were categorized)
train_root_files = os.listdir(dataset_train_path)
print(f"🔎 Images still in the train root folder: {len(train_root_files)}")
print(train_root_files[:10])  # Show first 10 images

🔎 Images still in the train root folder: 6
['acne', 'eksim', 'herpes', 'panu', 'rosacea', '_classes.csv']


In [55]:
# Count images in train and test root folders
train_images_root = [f for f in os.listdir(dataset_train_path) if f.lower().endswith(('png', 'jpg', 'jpeg'))]
test_images_root = [f for f in os.listdir(dataset_test_path) if f.lower().endswith(('png', 'jpg', 'jpeg'))]

print(f"🔎 Images still in Train root folder: {len(train_images_root)}")  
print(f"🔎 Images still in Test root folder: {len(test_images_root)}")  

🔎 Images still in Train root folder: 0
🔎 Images still in Test root folder: 0


In [57]:
for category in categories:
    train_files = os.listdir(os.path.join(dataset_train_path, category))
    test_files = os.listdir(os.path.join(dataset_test_path, category))
    print(f"📂 {category}: {len(train_files)} training images, {len(test_files)} test images")

📂 acne: 185 training images, 69 test images
📂 eksim: 200 training images, 65 test images
📂 herpes: 200 training images, 66 test images
📂 panu: 195 training images, 66 test images
📂 rosacea: 180 training images, 70 test images


In [59]:
##run2
# Define paths
dataset_train_path = r"C:\Users\avish\OneDrive\Desktop\Facial_Issues_Detector\train"
dataset_test_path = r"C:\Users\avish\OneDrive\Desktop\Facial_Issues_Detector\test"

In [61]:
# Define categories
categories = ['acne', 'eksim', 'herpes', 'panu', 'rosacea']

In [63]:
#run3
# Image Data Generators
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,  # Normalize images for MobileNetV2
    rotation_range=20,  
    width_shift_range=0.2,  
    height_shift_range=0.2,  
    shear_range=0.2,  
    zoom_range=0.2,  
    horizontal_flip=True,  
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)  # No augmentation

# Load images
train_generator = train_datagen.flow_from_directory(
    dataset_train_path,
    target_size=(224, 224),  # MobileNetV2 input size
    batch_size=32,
    class_mode="categorical"
)

test_generator = test_datagen.flow_from_directory(
    dataset_test_path,
    target_size=(224, 224),
    batch_size=32,
    class_mode="categorical"
)

print("✅ Dataset ready for training!")

Found 960 images belonging to 5 classes.
Found 336 images belonging to 5 classes.
✅ Dataset ready for training!


In [65]:
# Load the base model (pretrained on ImageNet)
base_model = MobileNetV2(input_shape=(224, 224, 3), include_top=False, weights='imagenet')

# Freeze all layers initially
base_model.trainable = False

# Add a custom classifier on top
x = base_model.output
x = GlobalAveragePooling2D()(x)  # Reduce dimensions
x = Dense(128, activation='relu')(x)
x = Dropout(0.4)(x)  # Prevent overfitting
x = Dense(len(categories), activation='softmax')(x)  # Corrected: Explicitly connect to x

# Create the final model
model = Model(inputs=base_model.input, outputs=x)  # Output should be x, not output

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()  # Display model architecture

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)      │ (None, 224, 224, 3)       │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ Conv1 (Conv2D)                │ (None, 112, 112, 32)      │             864 │ input_layer[0][0]          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ bn_Conv1 (BatchNormalization) │ (None, 112, 112, 32)      │             128 │ Conv1[0][0]                │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ Conv1_relu (ReLU)             │ (None, 112, 112, 32)      │               0 │ bn_Conv1[0][0]             │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ expanded_conv_depthwise       │ (None, 112, 112, 32)      │             288 │ Conv1_relu[0][0]           │
│ (DepthwiseConv2D)             │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ expanded_conv_depthwise_BN    │ (None, 112, 112, 32)      │             128 │ expanded_conv_depthwise[0… │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ expanded_conv_depthwise_relu  │ (None, 112, 112, 32)      │               0 │ expanded_conv_depthwise_B… │
│ (ReLU)                        │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ expanded_conv_project         │ (None, 112, 112, 16)      │             512 │ expanded_conv_depthwise_r… │
│ (Conv2D)                      │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ expanded_conv_project_BN      │ (None, 112, 112, 16)      │              64 │ expanded_conv_project[0][… │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block_1_expand (Conv2D)       │ (None, 112, 112, 96)      │           1,536 │ expanded_conv_project_BN[… │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block_1_expand_BN             │ (None, 112, 112, 96)      │             384 │ block_1_expand[0][0]       │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block_1_expand_relu (ReLU)    │ (None, 112, 112, 96)      │               0 │ block_1_expand_BN[0][0]    │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block_1_pad (ZeroPadding2D)   │ (None, 113, 113, 96)      │               0 │ block_1_expand_relu[0][0]  │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block_1_depthwise             │ (None, 56, 56, 96)        │             864 │ block_1_pad[0][0]          │
│ (DepthwiseConv2D)             │                           │               

 Total params: 2,422,597 (9.24 MB)

 Trainable params: 164,613 (643.02 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [67]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define ImageDataGenerator with augmentation for training data
train_datagen = ImageDataGenerator(
    rescale=1./255,       # Normalize pixel values
    rotation_range=20,    # Randomly rotate images
    width_shift_range=0.2, # Randomly shift width
    height_shift_range=0.2, # Randomly shift height
    shear_range=0.2,      # Apply shearing
    zoom_range=0.2,       # Random zoom
    horizontal_flip=True, # Flip images horizontally
    fill_mode='nearest'   # Fill missing pixels
)

# Only rescale for test data (no augmentation)
test_datagen = ImageDataGenerator(rescale=1./255)

# Define batch size
batch_size = 32

# Load training images
train_generator = train_datagen.flow_from_directory(
    dataset_train_path,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical'
)
# Load testing images
test_generator = test_datagen.flow_from_directory(
    dataset_test_path,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical'
)

Found 960 images belonging to 5 classes.
Found 336 images belonging to 5 classes.


In [69]:
# Train the model
history = model.fit(
    train_generator,
    epochs=20,  # Adjust based on performance
    validation_data=test_generator,
    steps_per_epoch=len(train_generator),
    validation_steps=len(test_generator)
)

C:\Users\avish\anaconda3\envs\py310_env\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
30/30 ━━━━━━━━━━━━━━━━━━━━ 62s 2s/step - accuracy: 0.3077 - loss: 1.7493 - val_accuracy: 0.5833 - val_loss: 1.1051
Epoch 2/20
30/30 ━━━━━━━━━━━━━━━━━━━━ 39s 1s/step - accuracy: 0.5515 - loss: 1.1592 - val_accuracy: 0.6458 - val_loss: 0.9332
Epoch 3/20
30/30 ━━━━━━━━━━━━━━━━━━━━ 38s 1s/step - accuracy: 0.6451 - loss: 0.9463 - val_accuracy: 0.6845 - val_loss: 0.8699
Epoch 4/20
30/30 ━━━━━━━━━━━━━━━━━━━━ 53s 2s/step - accuracy: 0.6729 - loss: 0.8443 - val_accuracy: 0.7202 - val_loss: 0.7607
Epoch 5/20
30/30 ━━━━━━━━━━━━━━━━━━━━ 47s 2s/step - accuracy: 0.7008 - loss: 0.7987 - val_accuracy: 0.7589 - val_loss: 0.6969
Epoch 6/20
30/30 ━━━━━━━━━━━━━━━━━━━━ 46s 2s/step - accuracy: 0.7364 - loss: 0.7459 - val_accuracy: 0.7440 - val_loss: 0.6584
Epoch 7/20
30/30 ━━━━━━━━━━━━━━━━━━━━ 46s 2s/step - accuracy: 0.7133 - loss: 0.7128 - val_accuracy: 0.7411 - val_loss: 0.6436
Epoch 8/20
30/30 ━━━━━━━━━━━━━━━━━━━━ 43s 1s/step - accuracy: 0.7459 - loss: 0.6752 - val_accuracy: 0.7649 - val_loss:

In [71]:
test_loss, test_accuracy = model.evaluate(test_generator)
print(f"✅ Test Accuracy: {test_accuracy:.4f}")
print(f"✅ Test Loss: {test_loss:.4f}")

11/11 ━━━━━━━━━━━━━━━━━━━━ 8s 684ms/step - accuracy: 0.8001 - loss: 0.4741
✅ Test Accuracy: 0.8006
✅ Test Loss: 0.4649


In [75]:
model.save(r"C:\Users\avish\OneDrive\Desktop\Facial_Issues_Detector\final_facial_issue_detector.keras")
print("✅ Best model saved successfully!")


✅ Best model saved successfully!


In [77]:
test_loss, test_acc = model.evaluate(test_generator)
print(f"✅ Final Test Accuracy: {test_acc:.4f}")
print(f"✅ Final Test Loss: {test_loss:.4f}")


11/11 ━━━━━━━━━━━━━━━━━━━━ 10s 749ms/step - accuracy: 0.8181 - loss: 0.4756
✅ Final Test Accuracy: 0.8006
✅ Final Test Loss: 0.4649


In [79]:
import cv2
import numpy as np
import tensorflow as tf

# Load the trained model
model = tf.keras.models.load_model("final_facial_issue_detector.keras")

# Define class labels (update as per your dataset)
class_labels = ["Acne", "Eksim", "Herpes", "Panu", "Rosacea"]

# Open webcam
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    # Get frame dimensions
    h, w, _ = frame.shape
    
    # Define oval region (center and size)
    center_x, center_y = w // 2, h // 2
    axes_x, axes_y = 150, 200  # Adjust oval size
    
    # Convert frame to grayscale for face detection
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    
    # Draw oval (black before detection, red when face is inside)
    oval_color = (0, 0, 0)  # Default Black
    for (x, y, w, h) in faces:
        if (center_x - axes_x < x < center_x + axes_x) and (center_y - axes_y < y < center_y + axes_y):
            oval_color = (0, 0, 255)  # Red if face inside oval
    
    cv2.ellipse(frame, (center_x, center_y), (axes_x, axes_y), 0, 0, 360, oval_color, 3)

    # Capture image when face is inside oval
    key = cv2.waitKey(1) & 0xFF
    if key == ord("c") and oval_color == (0, 0, 255):  # Press 'c' to capture
        face_img = frame[center_y - axes_y:center_y + axes_y, center_x - axes_x:center_x + axes_x]
        
        # Preprocess image for model
        face_img = cv2.resize(face_img, (224, 224))
        face_img = face_img / 255.0  # Normalize
        face_img = np.expand_dims(face_img, axis=0)  # Add batch dimension
        
        # Predict using model
        predictions = model.predict(face_img)
        predicted_class = class_labels[np.argmax(predictions)]
        confidence = np.max(predictions) * 100
        
        # Show results
        result_text = f"{predicted_class} ({confidence:.2f}%)"
        print(f"Detected: {result_text}")
        cv2.putText(frame, result_text, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    # Show video frame
    cv2.imshow("Facial Issue Detection", frame)

    # Press 'q' to quit
    if key == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
Detected: Eksim (88.42%)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
Detected: Eksim (82.87%)
